#### TYPE	DESCRIPTION
#### PER	Named person or family.
#### LOC	Name of politically or geographically defined location (cities, provinces, countries, international regions, bodies of water, mountains).
#### ORG	Named corporate, governmental, or other organizational entity.
#### MISC	Miscellaneous entities, e.g. events, nationalities, products or works of art.

In [88]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [8]:
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday JAVA for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


In [12]:
# !python -m spacy download it_core_news_sm
import it_core_news_sm
nlp_ita = it_core_news_sm.load()

In [22]:
doc = nlp_ita('Io sono Giorgio. Io vivo a Napoli. Python è un linguaggio di programmazione. Java. ')
print([(X.text, X.label_) for X in doc.ents])

[('Giorgio', 'PER'), ('Napoli', 'LOC'), ('Python', 'MISC'), ('Java', 'MISC')]


In [43]:
for i,j in dict([(X.text, X.label_) for X in doc.ents]).items():
    if j!='MISC':
        print(i)

Giorgio
Napoli


In [55]:
text = nlp_ita('io so usare bene Java e Python. ho qualche esperienza con R e ho fatto progetti di machine learning')
print([(X.text, X.label_) for X in text.ents])

[('Java', 'MISC'), ('Python', 'MISC'), ('R', 'MISC')]


## Test on the vacancies

In [57]:
# !pip install -r requirements.txt
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib
import re
import nltk
import numpy as np
from IPython.display import Image
import pickle
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('words')

In [86]:
professioni = pd.read_csv('data/professioni.csv')
Conoscenze = professioni.Conoscenze.apply(lambda x: str(x).split('\n'))
Conoscenze.iloc[0]

['LDAP',
 'LINQ',
 'MDX',
 'N1QL',
 'SPARQL',
 'XQuery',
 'business intelligence',
 'dati non strutturati',
 'valutazione della qualità dei dati']

In [98]:
text = nlp_ita(' '.join(Conoscenze.iloc[0]))
print([(X.text, X.label_) for X in text.ents])

[('SPARQL', 'MISC')]


In [101]:
subgroup = professioni.Subgroup.apply(lambda x: x.split('\n'))
subgroup.iloc[0]

['Attuario',
 'Data scientist',
 'Esperto in ricerca operativa',
 'Esperto ricerca operativa',
 'Statistici',
 'Statistico',
 'Statistico applicato',
 'Statistico economico',
 'Statistico esperto in controlli di qualità',
 'Statistico metodologico',
 '']

In [102]:
text = nlp_ita(' '.join(subgroup.iloc[0]))
print([(X.text, X.label_) for X in text.ents])

[('Data scientist Esperto', 'ORG'), ('Esperto', 'LOC'), ('Statistici Statistico Statistico', 'ORG'), ('Statistico economico Statistico', 'LOC'), ('Statistico', 'LOC')]


#### Does not work properly. Let's try something else